<a href="https://www.kaggle.com/code/thiagolimasantos/ps3-e17-eda-autogluon-models-lb-0-97882-top-6?scriptVersionId=134303406" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

!pip install autogluon.tabular
import os
import shutil
import subprocess
from collections import defaultdict
from copy import copy
from functools import partial
from itertools import product
from pathlib import Path

# Sub-modules and so on.
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import scipy.stats as stats
from colorama import Fore, Style
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import squareform
from scipy.stats import gaussian_kde
from scipy.stats import probplot
from IPython.core.display import HTML
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/machine-failure-predictions/machine failure.csv
/kaggle/input/playground-series-s3e17/sample_submission.csv
/kaggle/input/playground-series-s3e17/train.csv
/kaggle/input/playground-series-s3e17/test.csv
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 64.1 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29.76:
      Successfully uninstalled botocore-1.29.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.2

In [2]:

ON_KAGGLE = os.getenv("KAGGLE_KERNEL_RUN_TYPE") is not None
# Colorama settings.
CLR = (Style.BRIGHT + Fore.BLACK) if ON_KAGGLE else (Style.BRIGHT + Fore.WHITE)
RED = Style.BRIGHT + Fore.RED
BLUE = Style.BRIGHT + Fore.BLUE
CYAN = Style.BRIGHT + Fore.CYAN
RESET = Style.RESET_ALL

FONT_COLOR = "#010D36"
BACKGROUND_COLOR = "#F6F5F5"

CELL_HOVER = {  # for row hover use <tr> instead of <td>
    "selector": "td:hover",
    "props": "background-color: #F6F5F5",
}
TEXT_HIGHLIGHT = {
    "selector": "td",
    "props": "color: #FF2079; font-weight: bold",
}
INDEX_NAMES = {
    "selector": ".index_name",
    "props": "font-style: italic; background-color: #010D36; color: #F2F2F0;",
}
HEADERS = {
    "selector": "th:not(.index_name)",
    "props": "font-style: italic; background-color: #010D36; color: #F2F2F0;",
}
DF_STYLE = (INDEX_NAMES, HEADERS, TEXT_HIGHLIGHT)
# Html `code` block highlight.
HTML(
    """
<style>
code {
    background: rgba(58, 90, 129, 0.5) !important;
    border-radius: 4px !important;
    color: #f2f2f0 !important;
}
</style>
"""
)

In [3]:
############################### FEATURE ENGINER V2 #######################################################


# Read in the data
filepath = '/kaggle/input/playground-series-s3e17'

df_train = pd.read_csv(os.path.join(filepath, 'train.csv'), index_col=[0])
df_test = pd.read_csv(os.path.join(filepath, 'test.csv'), index_col=[0])
original = pd.read_csv('/kaggle/input/machine-failure-predictions/machine failure.csv', index_col=[0])
target_col = 'Machine failure'
# Stack training and original data
train = pd.concat([df_train, original])
full = pd.concat([df_train, original])
test = df_test.copy()

X_train = train.drop([f'{target_col}'],axis=1).reset_index(drop=True)
y_train = train[f'{target_col}'].reset_index(drop=True)
X_test = test.reset_index(drop=True)


def create_features(df):
    df["Temperature ratio"] = df['Process temperature [K]'] / df['Air temperature [K]']
    df['Power']= df['Torque [Nm]']*df['Rotational speed [rpm]']
    df['Torque * Tool wear'] = df['Torque [Nm]'] * df['Tool wear [min]']
    df['TotalFailures'] = df[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']].sum(axis=1)
    new_cols = [
        #'Temperature difference [K]', 
        'Power', 
        'Temperature ratio',
        'Torque * Tool wear','TotalFailures']
    return df, new_cols

def rename(df):
    df.columns = df.columns.str.replace('\[.*?\]', '', regex=True)
    df.columns = df.columns.str.strip()
    # df.columns = df.columns.str.replace(' ', '')
    
    return df

X_train, _ = create_features(X_train)
X_test, new_cols = create_features(X_test)
# Rename
X_train = rename(X_train)
X_test = rename(X_test)

In [4]:
full.head().style.set_table_styles(DF_STYLE).format(precision=3)

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,L50096,L,300.600,309.600,1596,36.100,140,0,0,0,0,0,0
1,M20343,M,302.600,312.100,1759,29.100,200,0,0,0,0,0,0
2,L49454,L,299.300,308.500,1805,26.500,25,0,0,0,0,0,0
3,L53355,L,301.000,310.900,1524,44.300,197,0,0,0,0,0,0
4,M24050,M,298.000,309.000,1641,35.400,34,0,0,0,0,0,0


In [5]:
# Check row duplicates
print(
    CLR + "Training Dataset Duplicated Rows:",
    RED + f"{full.drop('Machine failure', axis=1).duplicated().sum()}",
)

Training Dataset Duplicated Rows: 1186


In [6]:
color_map = [[0.0, "#01CBEE"], [0.5, "#010D36"], [1.0, "#FF2079"]]

pearson_corr = (
    df_train.drop("Machine failure", axis=1).corr(numeric_only=True, method="pearson").round(2)
)
mask = np.triu(np.ones_like(pearson_corr, dtype=bool))
lower_triangular_corr = (
    pearson_corr.mask(mask)
    .dropna(axis="index", how="all")
    .dropna(axis="columns", how="all")
)

heatmap = go.Heatmap(
    z=lower_triangular_corr,
    x=lower_triangular_corr.columns,
    y=lower_triangular_corr.index,
    text=lower_triangular_corr.fillna(""),
    texttemplate="%{text}",
    xgap=1,
    ygap=1,
    showscale=True,
    colorscale=color_map,
    colorbar_len=1.02,
    hoverinfo="none",
)
fig = go.Figure(heatmap)
fig.update_layout(
    font_color=FONT_COLOR,
    title="Correlation Matrix (Pearson) - Lower Triangular",
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    width=840,
    height=840,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange="reversed",
)
fig.show()

In [7]:
def get_kde_estimation(data_series):
    kde = gaussian_kde(data_series.dropna())
    kde_range = np.linspace(
        data_series.min() - data_series.max() * 0.1,
        data_series.max() + data_series.max() * 0.1,
        len(data_series),
    )
    estimated_values = kde.evaluate(kde_range)
    estimated_values_cum = np.cumsum(estimated_values)
    estimated_values_cum /= estimated_values_cum.max()
    return kde_range, estimated_values, estimated_values_cum


def get_n_rows_axes(n_features, n_cols=5, n_rows=None):
    n_rows = int(np.ceil(n_features / n_cols))
    current_col = range(1, n_cols + 1)
    current_row = range(1, n_rows + 1)
    return n_rows, list(product(current_row, current_col))

In [8]:
corr_threshold = 0.7

highest_abs_corr = (
    lower_triangular_corr.abs()
    .unstack()
    .sort_values(ascending=False)  # type: ignore
    .rename("Absolute Pearson Correlation")
)

highest_abs_corr = (
    highest_abs_corr[highest_abs_corr > corr_threshold]
    .to_frame()
    .reset_index(names=["Feature 1", "Feature 2"])
)

highest_corr_combinations = highest_abs_corr[["Feature 1", "Feature 2"]].to_numpy()
highest_abs_corr.style.set_table_styles(DF_STYLE).format(precision=2)

,Feature 1,Feature 2,Absolute Pearson Correlation
0,Air temperature [K],Process temperature [K],0.86
1,Rotational speed [rpm],Torque [Nm],0.78


In [9]:
full = rename(full)
feat = ['Air temperature', 'Process temperature',
       'Rotational speed', 'Torque']
numeric_data = full[feat]
numeric_cols = numeric_data#.drop("Machine_failure", axis=1).columns.tolist()
r2_scores = defaultdict(tuple)
def apply_transform(value):
    if value > 0:
        return value
    else:
        return np.nan
for feature in numeric_cols:
    orig = full[feature].dropna()
    orig = orig.map(apply_transform)  # Aplicando a verificação para os valores
    _, (*_, R_orig) = probplot(orig, rvalue=True)
    _, (*_, R_log) = probplot(np.log(orig), rvalue=True)
    _, (*_, R_sqrt) = probplot(np.sqrt(orig), rvalue=True)
    _, (*_, R_reci) = probplot(np.reciprocal(orig), rvalue=True)
    _, (*_, R_boxcox) = probplot(stats.boxcox(orig)[0], rvalue=True)
    _, (*_, R_yeojohn) = probplot(stats.yeojohnson(orig)[0], rvalue=True)
    r2_scores[feature] = (
        R_orig * R_orig,
        R_log * R_log,
        R_sqrt * R_sqrt,
        R_reci * R_reci,
        R_boxcox * R_boxcox,
        R_yeojohn * R_yeojohn,
    )
r2_scores = pd.DataFrame(
    r2_scores, index=("Original", "Log", "Sqrt", "Reciprocal", "BoxCox", "YeoJohnson")
).T

r2_scores["Winner"] = r2_scores.idxmax(axis=1)
r2_scores.style.set_table_styles(DF_STYLE).format(precision=3)

,Original,Log,Sqrt,Reciprocal,BoxCox,YeoJohnson,Winner
Air temperature,0.979,0.980,0.979,0.980,0.000,0.980,YeoJohnson
Process temperature,0.988,0.988,0.988,0.988,0.988,0.988,BoxCox
Rotational speed,0.882,0.934,0.911,0.000,0.998,0.998,BoxCox
Torque,1.000,0.960,0.990,0.760,1.000,1.000,BoxCox


In [10]:
from autogluon.tabular import TabularPredictor, TabularDataset
train = X_train
train["label"] = y_train
train = TabularDataset(train)

In [11]:
excluded_model_types = ['KNN', 'NN_TORCH']
predictor2 = TabularPredictor(label="label", problem_type='binary',verbosity=-1,
                             eval_metric='log_loss',sample_weight='balance_weight').fit(train,presets="best_quality",excluded_model_types=excluded_model_types,
                                                                    auto_stack=True,save_space=True,
                                                                    num_stack_levels=2, num_bag_folds=3,num_bag_sets=6,time_limit=120)
# ['accuracy', 'acc', 'balanced_accuracy', 'mcc', 'roc_auc_ovo_macro', 'log_loss', 'nll', 'pac', 'pac_score', 'quadratic_kappa', 'roc_auc', 'average_precision', 'precision', 'precision_macro', 'precision_micro',
#  'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_weighted']

In [12]:
predictor2.leaderboard()
#  -0.016092 

                 model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2  -0.091243       0.886658  15.515435                0.006857           1.936422            2       True          3
1    LightGBMXT_BAG_L1  -0.091260       0.566611   8.419741                0.566611           8.419741            1       True          1
2  WeightedEnsemble_L3  -0.095835       1.560591  26.906604                0.006901           1.919869            3       True          6
3      LightGBM_BAG_L2  -0.096400       1.157330  18.636519                0.277528           5.057506            2       True          5
4    LightGBMXT_BAG_L2  -0.103749       1.276163  19.929228                0.396361           6.350216            2       True          4
5      LightGBM_BAG_L1  -0.104000       0.313191   5.159271                0.313191           5.159271            1       True          2


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.091243,0.886658,15.515435,0.006857,1.936422,2,True,3
1,LightGBMXT_BAG_L1,-0.091260,0.566611,8.419741,0.566611,8.419741,1,True,1
2,WeightedEnsemble_L3,-0.095835,1.560591,26.906604,0.006901,1.919869,3,True,6
3,LightGBM_BAG_L2,-0.096400,1.157330,18.636519,0.277528,5.057506,2,True,5
4,LightGBMXT_BAG_L2,-0.103749,1.276163,19.929228,0.396361,6.350216,2,True,4
5,LightGBM_BAG_L1,-0.104000,0.313191,5.159271,0.313191,5.159271,1,True,2


In [13]:
#y_pred_auc = predictor.predict_proba(X_test, as_pandas=False, as_multiclass=False)
y_pred_logloss = predictor2.predict_proba(X_test, as_pandas=False, as_multiclass=False)

In [14]:
y_pred_logloss

array([0.00041207, 0.00048911, 0.00034417, ..., 0.00059131, 0.00359378,
       0.00039809])

In [15]:

#models = ['WeightedEnsemble_L3','WeightedEnsemble_L2']

# Inicialize uma matriz vazia para armazenar as previsões de cada modelo
#predictions = []

# Faça previsões para cada modelo
#for models in models:
    # Realize a previsão para o modelo atual
#    model_pred = predictor.predict_proba(X_test, model=models,as_pandas=False, as_multiclass=False)
    # Adicione as previsões do modelo atual à matriz
#    predictions.append(model_pred)

# Calcule a média das previsões de todos os modelos
#average_prediction = np.mean(predictions, axis=0)

# Exiba a média das previsões
#print("Average Prediction:", average_prediction)
#Average Prediction: [0.03733805 0.06888288 0.0210157  ... 0.04945559 0.02032589 0.03096515]

In [16]:


#ypredmedia = (y_pred_auc+y_pred_logloss+average_prediction)/3
def make_submission(test_predss, prefix=''):
    sub = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
    sub[f'{target_col}'] = test_predss
    sub.to_csv(f'{prefix}submission.csv', index=False)
    return  sub
#sub = make_submission(np.where(ypredmedia >= 0.50, 1, 0), prefix='')
sub = make_submission(y_pred_logloss, prefix='')
sub

,id,Machine failure
0,136429,0.000412
1,136430,0.000489
2,136431,0.000344
3,136432,0.000516
4,136433,0.000764
...,...,...
90949,227378,0.000492
90950,227379,0.023965
90951,227380,0.000591
90952,227381,0.003594
